### The purpose of this tutorial is to walk through the execution of this code, starting with inputs and calculating the displacements and forces on the matrix defined. 

In [ ]:
#Import Libraries

import numpy as np
import matplotlib.pyplot as plt
from src import *

In [ ]:
# Define inputs
E = 10000  # Young's Modulus (Pa)
nu = 0.3  # Poisson's Ratio
#Radius and Area
r = 1 
A = np.pi * r**2 
#Moments of Inertia
Iy = np.pi * r**4 / 4
Iz = np.pi * r**4 / 4
Irho = np.pi * r**4 / 2
J = np.pi * r**4 / 2

P = 1
L = np.sqrt(((18)**2) + ((56)**2) + ((44)**2))
Fx = -1*P*(18)/L
Fy = -1*P*(56)/L
Fz = -1*P*(44)/L

#Define Nodes (x,y,z, node_id)
nodes = np.array([
    [0.0, 0.0, 0.0, 0],
    [18.0*(1/6), 56*(1/6), 44*(1/6), 1],
    [18.0*(2/6), 56*(2/6), 44*(2/6), 2],
    [18.0*(3/6), 56*(3/6), 44*(3/6), 3],
    [18.0*(4/6), 56*(4/6), 44*(4/6), 4],
    [18.0*(5/6), 56*(5/6), 44*(5/6), 5],  
    [18.0, 56.0, 44.0, 6], 
])

#Connect the Nodes
elements = np.array([
    [0, 1],
    [1, 2], 
    [2, 3],  
    [3, 4], 
    [4, 5], 
    [5, 6], 
])

#Assign constraints
# (0.0, 0.0, 0.0, 0.0, 0.0, 0.0) = fixed node
# (0.0, 0.0, 0.0, None, None, None) = pinned node
# (None, None, None, None, None, None) = free node
fixed_nodes = {
    0: (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 
    1: (None, None, None, None, None, None),
    2: (None, None, None, None, None, None), 
    3: (None, None, None, None, None, None),
    4: (None, None, None, None, None, None), 
    5: (None, None, None, None, None, None),
    6: (None, None, None, None, None, None),
}
print (Fx, Fy, Fz)
loads = {
    6: (-0.2450397, -0.7623458, -0.598986, 0.0, 0.0, 0.0),
}

### Compute Displacements and Forces


In [ ]:
# --- Analysis and Plotting ---
# Set up boundary conditions
beam = Beam(nodes, elements, E, nu, A, Iy, Iz, Irho, J)
bc = BoundaryCondition(fixed_nodes)
for node_id, load in loads.items():
    bc.apply_load(node_id, load)

# Run the analysis
solver = BeamSolver(beam, bc)
displacements, reactions = solver.solve()
print("Static Analysis Results:")
solver.display_results(displacements, reactions) 
print("Internal Forces per Element:")
for elem_id, forces in solver.internal_forces.items():
    print(f"Element {elem_id}: {np.round(forces, 5)}")



### Elastic Critical Mode Analysis

In [ ]:
# Buckling analysis
eigvals, eigvecs, buckling_forces = solver.solve_buckling()

eigenvals = eigvals [:3]
print ('Elastic Critical Load Factor', eigenvals)

# Generate plot
plot_buckling_mode(beam, eigvecs, buckling_forces, eigvals)